# <center> 2.9 А нейронки будут? </center>

In [52]:
import numpy as np
import pandas as pd

In [53]:
# Загружаем train-датасет который мы сохранили на шаге quickstart
rides_info = pd.read_csv('../data/quickstart_train.csv')
rides_info.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,15634.32,another_bug,44.660920,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,2791.94,electro_bug,49.862069,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,2263.93,gear_stick,44.005747,1.588366e+07,0.1,102.382857,2.513319,174,173
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,1518.06,engine_fuel,49.344828,1.651883e+07,0.1,172.793237,-5.029476,174,170
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,825.78,engine_fuel,50.603448,1.398317e+07,0.1,203.462289,-14.260456,174,171


In [54]:
drop_cols = ['car_id', 'target_reg', 'target_class']
cat_cols = ['car_type', 'fuel_type', 'model']

In [55]:
# закодируем категориальные фичи в one hot encoding вектора
rides_info = pd.get_dummies(rides_info, columns=cat_cols)

In [56]:
# заполним пропущенные значения медианным значением по столбцу
rides_info.fillna(rides_info.median(), inplace=True)

/tmp/ipykernel_838/2067634496.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  rides_info.fillna(rides_info.median(), inplace=True)


<h1><center>Для начала возьмем готовую реализацию нейронной сети MLP из sklearn</center></h1>

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [58]:
X = rides_info.drop(drop_cols, axis=1)
y = rides_info['target_class']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **Классификация**

In [14]:
clf = MLPClassifier(random_state=42, max_iter=100, verbose=True).fit(X_train, y_train)

Iteration 1, loss = 31.99382509
Iteration 2, loss = 32.15449761
Iteration 3, loss = 31.99382393
Iteration 4, loss = 32.28309820
Iteration 5, loss = 32.36023759
Iteration 6, loss = 31.90499338
Iteration 7, loss = 32.10967484
Iteration 8, loss = 32.22524094
Iteration 9, loss = 31.72383062
Iteration 10, loss = 31.90110747
Iteration 11, loss = 32.24452460
Iteration 12, loss = 31.81438172
Iteration 13, loss = 31.70454408
Iteration 14, loss = 31.64668880
Iteration 15, loss = 32.18666835
Iteration 16, loss = 32.22755536
Iteration 17, loss = 31.43424254
Iteration 18, loss = 31.80096785
Iteration 19, loss = 32.10782103
Iteration 20, loss = 32.48528058
Iteration 21, loss = 32.41971144
Iteration 22, loss = 31.76250559
Iteration 23, loss = 32.30481520
Iteration 24, loss = 31.89739216
Iteration 25, loss = 32.10952705
Iteration 26, loss = 31.80503258
Iteration 27, loss = 31.74311212
Iteration 28, loss = 31.93596205
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopp

In [15]:
preds = clf.predict(X_test)

In [16]:
preds[:5]

array(['electro_bug', 'engine_overheat', 'wheel_shake', 'wheel_shake',
       'engine_overheat'], dtype='<U15')

In [17]:
f1_score(y_test, preds, average='macro')

0.030400788242116648

## **Регрессия**

In [18]:
y = rides_info['target_reg']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
regr = MLPRegressor(random_state=42).fit(X_train, y_train)

In [21]:
preds = regr.predict(X_test)
preds[:5]

array([3063.34633881, 1977.44798248,  741.31151105, 2635.05890012,
       1064.84778015])

In [22]:
mean_squared_error(y_test, preds)

31723160.539891228

## Теперь попробуем решить задачу с помощью самописного MLP с двумя головами. На фрэймворке PyTorch 

In [59]:
rides_df = rides_info.drop(columns=['car_id'])

In [60]:
# переведем строковые значения категориального таргета в целочисленные
le = LabelEncoder()
rides_df['target_class'] = le.fit_transform(rides_df['target_class'])

### GaussRankScaler - SOTA scaler для нейросетей, показывает лучший скор, чем скейлеры из sklearn

In [61]:
# для нормализации данных используем ранк гаусс трансформацию. 
# импортнем библеотеку и добавим путь к ней 
#!git clone https://github.com/aldente0630/gauss-rank-scaler
import sys
sys.path.append('./gauss-rank-scaler')

In [62]:
# числовые переменные, которые подвергнем трансформации
num_cols = [col for col in list(rides_df.columns)
            if col not in ['target_reg', 'target_class']]

In [63]:
from gauss_rank_scaler import GaussRankScaler

scaler = GaussRankScaler()
df = scaler.fit_transform(rides_df[num_cols])

/home/jovyan/storage/Aleron/Course/Competitive_Data_Science/notebooks/./gauss-rank-scaler/gauss_rank_scaler.py:79: RuntimeWarning: invalid value encountered in divide
  scaled_rank = np.clip(rank / factor - self.bound, -self.bound, self.bound)


In [64]:
df = pd.DataFrame(df, columns=num_cols)

In [65]:
target_scaler = GaussRankScaler()
target = target_scaler.fit_transform(rides_df['target_reg'].values.reshape(-1, 1))

In [66]:
df['target_reg'] = target
df['target_class'] = rides_df['target_class']

In [67]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [68]:
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
import os
import copy

In [69]:
# для наших данных и размера нейросети подойдет запуск на cpu
device = torch.device('cpu')

In [70]:
# ВАЖНО! - фиксируем воспроизводимость
def seed_everything(seed=42):
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [83]:
# можно попробовать поэкспериментировать с параметрами
class CFG:
    hidden_size=128
    dropout=0.1
    lr=1e-3
    batch_size=128
    num_workers=4
    epochs=20
    num_features=train.shape[1]-2 # кол-во фичей подаваемое на вход
    num_tar_class=train.target_class.nunique() # количество выходов равно кол-ву предсказываемых классов

In [84]:
# датасет выдает фичи и значения целевых переменных
class Rides(Dataset):
    
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx,:]
        
        data = row.drop(labels=['target_reg', 'target_class'])
        
        data = torch.FloatTensor(data.values.astype('float'))
        tar_reg = torch.tensor(row['target_reg']).float()
        tar_class = row['target_class'].astype('int')
        
        return data, tar_reg, tar_class

In [85]:
# проверим корректность работы методов нашего класса
next(iter(Rides(train)))

(tensor([-0.3436, -2.7511, -1.6442,  2.7511,  0.0345, -0.1473, -2.7511,  1.9094,
         -0.2307,     nan,  1.0498, -2.7511,  2.7511, -2.7511, -2.7511, -2.7511,
          2.7511, -2.7511, -2.7511, -2.7511, -2.7511, -2.7511, -2.7511, -2.7511,
         -2.7511,  2.7511, -2.7511, -2.7511, -2.7511, -2.7511, -2.7511, -2.7511,
         -2.7511, -2.7511, -2.7511, -2.7511, -2.7511, -2.7511, -2.7511, -2.7511,
         -2.7511, -2.7511, -2.7511]),
 tensor(-0.8442),
 2)

In [86]:
train_datasets = {'train': Rides(train),
                  'val': Rides(test)}

In [87]:
dataloaders_dict = {x: torch.utils.data.DataLoader(train_datasets[x], 
                                                   batch_size=CFG.batch_size, 
                                                   shuffle=True, 
                                                   num_workers=CFG.num_workers)
                    for x in ['train', 'val']}

In [88]:
# Построим архитектуру mlp с двумя головами для регрессии и классифкации

class TabularNN(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.mlp = nn.Sequential(
                          nn.Linear(cfg.num_features, cfg.hidden_size),
                          #nn.BatchNorm1d(cfg.hidden_size),
                          nn.Dropout(cfg.dropout),
                          nn.ReLU(),
                          nn.Linear(cfg.hidden_size, cfg.hidden_size),
                          #nn.BatchNorm1d(cfg.hidden_size),
                          nn.Dropout(cfg.dropout),
                          nn.ReLU(),
                          nn.Linear(cfg.hidden_size, cfg.hidden_size//2),
                          )
        
        self.regressor = nn.Sequential(
            nn.Linear(cfg.hidden_size//2, 1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(cfg.hidden_size//2, cfg.num_tar_class)
        )

    def forward(self, data):
        x = self.mlp(data)
        tar_reg = self.regressor(x)
        tar_class = self.classifier(x)
        return tar_reg.view(-1), tar_class

In [89]:
model = TabularNN(CFG).to(device)

In [90]:
# оптимайзер и лоссы для регрессии и классификации
optimizer = torch.optim.Adam(model.parameters(), lr = CFG.lr)
regression_criterion = nn.MSELoss().to(device)
classification_criterion = nn.CrossEntropyLoss().to(device)

In [91]:
# функция для тренировки, на выходе - обученная модель
def train_model(model, dataloaders, regression_criterion,
                classification_criterion, optimizer, num_epochs=25):

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100000.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for inputs, labels_1, labels_2 in dataloaders[phase]:
                inputs = inputs.to(device)
                labels_1 = labels_1.to(device)
                labels_2 = labels_2.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss

                    outputs_1, outputs_2 = model(inputs)
                    loss_1 = regression_criterion(outputs_1, labels_1)
                    loss_2 = classification_criterion(outputs_2, labels_2)

                    loss = loss_1 + loss_2

                    _, preds_2 = torch.max(outputs_2, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                val_acc_history.append(running_loss)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_model_wts = copy.deepcopy(model.state_dict())


    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [92]:
model_ft = train_model(model, dataloaders_dict, regression_criterion,
                classification_criterion, optimizer, num_epochs=22)

Epoch 0/21
----------
train Loss: nan
val Loss: nan
Epoch 1/21
----------
train Loss: nan
val Loss: nan
Epoch 2/21
----------
train Loss: nan
val Loss: nan
Epoch 3/21
----------
train Loss: nan
val Loss: nan
Epoch 4/21
----------
train Loss: nan
val Loss: nan
Epoch 5/21
----------
train Loss: nan
val Loss: nan
Epoch 6/21
----------
train Loss: nan
val Loss: nan
Epoch 7/21
----------
train Loss: nan
val Loss: nan
Epoch 8/21
----------
train Loss: nan
val Loss: nan
Epoch 9/21
----------
train Loss: nan
val Loss: nan
Epoch 10/21
----------
train Loss: nan
val Loss: nan
Epoch 11/21
----------
train Loss: nan
val Loss: nan
Epoch 12/21
----------
train Loss: nan
val Loss: nan
Epoch 13/21
----------
train Loss: nan
val Loss: nan
Epoch 14/21
----------
train Loss: nan
val Loss: nan
Epoch 15/21
----------
train Loss: nan
val Loss: nan
Epoch 16/21
----------
train Loss: nan
val Loss: nan
Epoch 17/21
----------
train Loss: nan
val Loss: nan
Epoch 18/21
----------
train Loss: nan
val Loss: nan
Epo

In [93]:
# p1, p2 - предсказания; l1, l2  - истинные значения

p1 = []
p2 = []
l1 = []
l2 = []

with torch.set_grad_enabled(False):
    # Get model outputs and calculate loss
    for inputs, labels_1, labels_2 in dataloaders_dict['val']:
        inputs = inputs.to(device)
        labels_1 = labels_1.to(device)
        labels_2 = labels_2.to(device)
        l1.extend(labels_1.detach().cpu().numpy())
        l2.extend(labels_2.detach().cpu().numpy())
        
        outputs_1, outputs_2 = model_ft(inputs)
        _, outputs_2 = torch.max(outputs_2, 1)

        p1.extend(outputs_1.detach().cpu().numpy())
        p2.extend(outputs_2.detach().cpu().numpy())

**посмотрим на результаты по каждой из задач**

In [94]:
# классификация
from sklearn.metrics import f1_score
f1_score(l2, p2, average='macro')

0.024503591043515

In [95]:
p2 = target_scaler.inverse_transform(np.array(p2).reshape(-1, 1))
l2 = target_scaler.inverse_transform(np.array(l2).reshape(-1, 1))

In [96]:
# регрессия
mean_squared_error(l2, p2)

14483562335.115906

чтобы улучшить скор можно поэкспериментировать с: 
- нормализацией данных, заполнением пропусков
- архитектурой сети
- попробовать lr sheduler

# А теперь решим задачу с помощью TABNET

https://arxiv.org/abs/1908.07442

In [97]:
X = rides_info.drop(drop_cols + ['target_reg', 'target_class'], axis=1)
y = rides_info['target_class']

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [100]:
pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 885.9 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [101]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

## **Классификация**

In [102]:
clf = TabNetClassifier(device_name='cpu')
clf.fit(
  X_train.values, y_train,
  eval_set=[(X_test.values, y_test)]
)

/opt/conda/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.86534 | val_0_accuracy: 0.11752 |  0:00:00s
epoch 1  | loss: 2.73252 | val_0_accuracy: 0.10897 |  0:00:00s
epoch 2  | loss: 2.57215 | val_0_accuracy: 0.09402 |  0:00:00s
epoch 3  | loss: 2.46832 | val_0_accuracy: 0.10043 |  0:00:00s
epoch 4  | loss: 2.32645 | val_0_accuracy: 0.09829 |  0:00:00s
epoch 5  | loss: 2.43553 | val_0_accuracy: 0.1047  |  0:00:00s
epoch 6  | loss: 2.29796 | val_0_accuracy: 0.10043 |  0:00:00s
epoch 7  | loss: 2.28731 | val_0_accuracy: 0.1047  |  0:00:00s
epoch 8  | loss: 2.26514 | val_0_accuracy: 0.1047  |  0:00:00s
epoch 9  | loss: 2.23654 | val_0_accuracy: 0.1047  |  0:00:00s
epoch 10 | loss: 2.21712 | val_0_accuracy: 0.1047  |  0:00:00s

Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_0_accuracy = 0.11752


/opt/conda/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [117]:
preds = clf.predict(X_test.values)

In [41]:
preds[:5]

array(['engine_fuel', 'another_bug', 'another_bug', 'another_bug',
       'engine_check'], dtype='<U15')

In [118]:
f1_score(y_test, preds, average='macro')

0.5659570480334537

## **Регрессия**

In [119]:
y = rides_info['target_1']

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [121]:
reg = TabNetRegressor(device_name='cpu')
reg.fit(
  X_train.values, y_train.values.reshape(-1,1),
  eval_set=[(X_test.values, y_test.values.reshape(-1,1))]
)

/opt/conda/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 13071166116457.59| val_0_mse: 13114763854235.57|  0:00:22s
epoch 1  | loss: 13036448048594.746| val_0_mse: 13082107736203.969|  0:00:44s
epoch 2  | loss: 12993154211911.0| val_0_mse: 13012934553076.578|  0:01:06s
epoch 3  | loss: 12938276803404.754| val_0_mse: 12971329225959.338|  0:01:28s
epoch 4  | loss: 12864850954122.873| val_0_mse: 12913032110380.582|  0:01:50s
epoch 5  | loss: 12774357662267.453| val_0_mse: 12762433003363.947|  0:02:12s
epoch 6  | loss: 12660569340475.453| val_0_mse: 12629622799360.06|  0:02:34s
epoch 7  | loss: 12509594837346.947| val_0_mse: 12383024418846.844|  0:02:56s
epoch 8  | loss: 12299034565051.672| val_0_mse: 12068202789007.113|  0:03:19s
epoch 9  | loss: 12039692082078.393| val_0_mse: 11826439971454.65|  0:03:41s
epoch 10 | loss: 11749502732085.686| val_0_mse: 11670502449324.525|  0:04:03s
epoch 11 | loss: 11406910236767.844| val_0_mse: 11287346087265.936|  0:04:25s
epoch 12 | loss: 11041688805333.412| val_0_mse: 10348520529247.09|  0:

/opt/conda/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [125]:
preds = reg.predict(X_test.values)

In [126]:
preds[:5]

array([[5160755.5],
       [1466021.5],
       [1387069.1],
       [4700907. ],
       [1508347.6]], dtype=float32)

In [127]:
mean_squared_error(y_test.values.reshape(-1,1), preds)

2727736816756.805

# <center> Трансформеры </center>

![sheet](./images/tr.png) ![sheet](./images/hf.png)

## Библиотека transformers и платформа Hugging Face

Отличный туториал для начинающих: <a> https://habr.com/ru/post/704592/ </a>